In [ ]:
# MOCK TESTS WITH PYTEST
# 

from unittest import mock

import pytest
from typing import TYPE_CHECKING, Literal, Optional, Union
from src.DAO.movie_dao import MovieDAO
from datetime import date
from src.Model.movie import Movie
from tests.conftest import avatar  # Assurez-vous que `avatar` est bien défini ici

class MockMovieDBConnector:
    def __init__(self, movie=None):
        self.movie = movie or avatar  # Utilise l'avatar si aucun film n'est fourni

    def sql_query(
        self,
        query: str,
        data: Optional[Union[tuple, list, dict]] = None,
        return_type: Union[Literal["one"], Literal["all"]] = "one",
    ):
        match query:
            case "SELECT * FROM movie WHERE id_movie=%s":
                if not data:
                    raise Exception("No data provided.")
                id_movie = data[0]  # Récupère l'ID à partir de la donnée
                if id_movie == 19995:
                    return Movie(**self.movie)
        return None  # Pour d'autres requêtes non définies

@pytest.fixture
def mock_db_connection():
    return MockMovieDBConnector()

def test_get_movie_by_id(mock_db_connection):
    movie_dao = MovieDAO(mock_db_connection)
    movie = movie_dao.get_by_id(19995)
    
    assert movie is not None
    assert movie.id_movie == 19995
    assert movie.title == "Avatar"
    assert movie.adult is False
    assert movie.budget == 237000000

def test_get_movie_by_non_existent_id(mock_db_connection):
    movie_dao = MovieDAO(mock_db_connection)
    movie = movie_dao.get_by_id(99999)  # ID qui n'existe pas
    
    assert movie is None  # Vérifiez que la méthode renvoie None pour un film inexistant



In [2]:
from typing import Annotated

from fastapi import APIRouter, Depends, HTTPException, status
from fastapi.security import HTTPAuthorizationCredentials

from src.Model.connected_user import ConnectedUser
from src.Model.movie import Movie
from src.Webservice.init_app import (
    recommend_service,
    user_dao,
    movie_dao,
    user_favorite_dao,
    user_follow_dao,
    user_interaction_service,
)

current_user = user_dao.get_user_by_id(431)
users = [user_dao.get_user_by_id(id).to_dict() for id in current_user.follow_list]
users

[{'username': 'xgarza', 'own_film_collection': None, 'follow_list': None},
 {'username': 'figueroapaul',
  'own_film_collection': None,
  'follow_list': None},
 {'username': 'thomasedwards',
  'own_film_collection': [388, 364],
  'follow_list': [328, 301, 349, 284, 323]},
 {'username': 'elizabeth84', 'own_film_collection': None, 'follow_list': None}]

In [10]:
comments = [1,None,None,2]
comments = [x for x in comments if x != None]
print(comments)


[1, 2]


In [2]:
from datetime import datetime
from typing import List

from src.DAO.db_connection import DBConnector

dao = DBConnector()
query = """
    INSERT INTO comment (id_user, id_movie, comment, date)
    VALUES (%s, %s, %s, %s);
    """
dao.sql_query(query, (100,200,"je l'adore",datetime.now()))

AttributeError: 'DBConnector' object has no attribute 'db_connection'

In [3]:
a=[1,4]
b=[1]
list(set(a+b))

[1, 4]

In [3]:
from src.Service.password_service import hash_password, create_salt

password = "NoCode"
salt = ["tes", "tuser", "2"]
hashed_password = hash_password(password)
print(hashed_password)    

9da90a168c5026a3291f7612de2caebc7e36ae4c630416fc3332a41a1c630fee


In [ ]:
from src.Model.connected_user import ConnectedUser
from src.Model.genre import Genre
from src.Model.movie import Movie
from datetime import date
d = Movie(
                id_movie=1,
                title="Movie 1",
                belongs_to_collection=None,
                budget=1000000,
                genres=[Genre(id=123, name="Musical Comedy")],
                origin_country=["US"],
                original_language="en",
                original_title="Movie 1",
                overview="Overview 1",
                popularity=8.5,
                release_date=date(2023, 1, 1),
                revenue=5000000,
                runtime=120,
                vote_average=7.5,
                vote_count=1000,
                adult=False,
            )
print (d)

Title : Movie 1, ( ID: 1 ), Release Date: 2023-01-01, Popularity: 8.5, Vote Average: 7.5, Vote Count: 1000


In [ ]:
result= [
            {
                "id_user": 2,
                "username": "user2",
                "hashed_password": "hashed2",
                "date_of_birth": date(1992, 2, 2),
                "gender": 2,
                "first_name": "First2",
                "last_name": "Last2",
                "email_address": "user2@example.com",
                "password_token": "token2",
                "phone_number": "0987654321",
                "own_film_collection": [2],
                "follow_list": [1],
            }
        ]
user = dict(result)
print(ConnectedUser(**user))

ValueError: dictionary update sequence element #0 has length 12; 2 is required

In [3]:
import pandas as pd
import numpy as np

# Define the schema based on the provided mermaid code
schema = {
    "movie": ["id_movie", "title", "budget", "origin_country", "original_language", "original_title", "overview", "popularity", "release_date", "revenue", "runtime", "vote_average", "vote_count", "adult"],
    "genre": ["id_genre", "name_genre"],
    "movie_collection": ["id_movie_collection", "name_movie_collection"],
    "link_movie_movie_collection": ["id_movie", "id_movie_collection"],
    "link_movie_genre": ["id_movie", "id_genre"],
    "movie_maker": ["id_movie_maker", "adult", "name", "gender", "biography", "birthday", "place_of_birth", "deathday", "known_for_department", "popularity"],
    "KnownFor": ["id_movie", "id_movie_maker"],
    "users": ["id_user", "username", "first_name", "last_name", "password_token", "hashed_password", "email_address", "date_of_birth", "phone_number", "gender"],
    "rating": ["id_user", "id_movie", "rating", "date"],
    "comment": ["id_user", "id_movie", "comment", "date"],
    "follower": ["id_user", "id_user_followed", "date"]
}

# Function to generate random data for each table
def generate_data(table_name, columns, num_rows=100):
    data = {}
    for column in columns:
        if column == "id_user":
            data[column] = [i in range(1, num_rows + 1)]
        elif column == "id_movie":
            data[column] = [i in range(1, num_rows + 1)]
        elif column == "id_genre":
            data[column] = [i in range(1, num_rows + 1)]
        elif column == "title":
            data[column] = [f"movie_{i}" for i in range(1, num_rows + 1)]
        elif column == "budget":
            data[column] = np.random.uniform(100000, 100000000, num_rows)
        elif column == "origin_country":
            countries = ["USA", "UK", "France", "Germany","China", "Brasil", "Spain", "Japon"]
            data[column] = np.random.choice(countries, num_rows)
        elif column == "original_language":
            languages = ["English", "French", "German"]
            data[column] = np.random.choice(languages, num_rows)
        elif column == "original_title":
            data[column] = [f"original_title_{i}" for i in range(1, num_rows + 1)]
        elif column == "overview":
            data[column] = [f"overview_{i}" for i in range(1, num_rows + 1)]
        elif column == "popularity":
            data[column] = np.random.uniform(0, 100, num_rows)
        elif column == "release_date":
            data[column] = pd.date_range(start='1/1/2000', periods=num_rows, freq='M')
        elif column == 'revenue':
            data[column] = np.random.randint(1000000, 1000000000, num_rows)
        elif column == 'runtime':
            data[column] = np.random.randint(80, 180, num_rows)
        elif column == 'vote_average':
            data[column] = np.random.uniform(0, 10, num_rows)
        elif column == 'vote_count':
            data[column] = np.random.randint(0, 10000, num_rows)
        elif column == 'adult':
            data[column] = np.random.choice([False], num_rows)
        elif column == 'name_genre':
            genres = ["Action", "Comedy", "Drama", "Horror", "Sci-Fi","Love","revenge","death","Devils"]
            data[column] = np.random.choice(genres, num_rows)
        elif column == 'name_movie_collection':
            data[column] = [f"collection_{i}" for i in range(1, num_rows + 1)]
        elif column == 'name':
            data[column] = [f"name_{i}" for i in range(1, num_rows + 1)]
        elif column == 'gender':
            data[column] = np.random.choice([0, 1], num_rows)
        elif column == 'biography':
            data[column] = [f"biography_{i}" for i in range(1, num_rows + 1)]
        elif column == 'birthday' or column == 'deathday' or column == 'date_of_birth' or column == 'date':
            data[column] = pd.date_range(start='1/1/1950', periods=num_rows, freq='Y')
        elif column == 'place_of_birth':
            places = ["New York", "London", "Paris"]
            data[column] = np.random.choice(places, num_rows)
        elif column == 'known_for_department':
            departments = ["Directing", "Acting", "Writing"]
            data[column] = np.random.choice(departments, num_rows)
        elif column == 'popularity':
            data[column] = np.random.uniform(0, 100, num_rows)
        elif column == 'known_for':
            known_for_list = [{"movie_id": np.random.randint(1, num_rows)} for _ in range(num_rows)]
            data[column] = known_for_list
        elif column == 'username':
            data[column] = [f"user_{i}" for i in range(1, num_rows + 1)]
        elif column == 'first_name':
            first_names = ["John", "Jane", "Alice", "Bob"]
            data[column] = np.random.choice(first_names, num_rows)
        elif column == 'last_name':
            last_names = ["Doe", "Smith", "Johnson"]
            data[column] = np.random.choice(last_names, num_rows)
        elif column == 'password_token' or column == 'hashed_password':
            data[column] = [f"token_{i}" for i in range(1, num_rows + 1)]
        elif column == 'email_address':
            data[column] = [f"user_{i}@example.com" for i in range(1, num_rows + 1)]
        elif column == 'phone_number':
            phone_numbers = [f"+123456789{i}" for i in range(num_rows)]
            data[column] = phone_numbers
        else:
            data[column] = np.random.randint(1, num_rows + 1, num_rows)
    return pd.DataFrame(data)

# Generate data for each table
dataframes = {table: generate_data(table, columns) for table, columns in schema.items()}

# Save each dataframe to a separate sheet in a Google Sheets workbook
with pd.ExcelWriter("simulated_data.xlsx") as writer:
    for table, df in dataframes.items():
        df.to_excel(writer, sheet_name=table, index=False)

print("Google Sheets workbook 'simulated_data.xlsx' with simulated data has been created.")


Google Sheets workbook 'simulated_data.xlsx' with simulated data has been created.


In [4]:
dataframes.head()

AttributeError: 'dict' object has no attribute 'head'